In [6]:
%pip install dulwich

In [2]:
import time
import requests

from IPython.display import display, HTML


def log_to_notebook(message):
    display(HTML(message))


response = requests.post(
    "https://github-auth-proxy.onrender.com/get-device-code",
    headers={"Content-Type": "application/json"},
)

if response.status_code != 200:
    raise Exception("Network response was not ok")

data = response.json()
device_code = data["device_code"]
log_to_notebook(
    f'<br><strong>1. Visit:</strong> <a href="{data["verification_uri"]}" target="_blank">{data["verification_uri"]}</a>'
)
log_to_notebook(f"2. Enter the code below")
print(data["user_code"])

9A8F-9CEB


In [3]:
token_response = requests.post(
    "https://github-auth-proxy.onrender.com/exchange-token",
    headers={"Content-Type": "application/json"},
    json={"code": device_code},
)

token_data = token_response.json()
if token_data.get("access_token"):
    print("Authentication successful!")
    github_access_token = token_data["access_token"]
elif token_data.get("error") == "authorization_pending":
    print("Waiting for user approval...")
    time.sleep(data["interval"])
else:
    print("Error: " + token_data.get("error", "Unknown error"))

Authentication successful!


In [29]:
!rm -fr github_auth_proxy_render

In [11]:
token_file_path = ".github_access_token"
with open(token_file_path, "w") as token_file:
    token_file.write(github_access_token)
print(f"Token saved to {token_file_path}")

Token saved to .github_access_token
